In [4]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import download_data
import load_data
from column_checks import get_dtypes
from filepaths import *
import impute_hourly_profiles
import data_cleaning
import output_data
import emissions
import validation
import gross_to_net_generation
import eia930
from logging_util import get_logger, configure_root_logger
configure_root_logger()
logger = get_logger("test")


year = 2021
path_prefix = f"{year}/"

In [1]:
dtypes_to_use = {
        "plant_id_eia": "Int32",
        "plant_id_epa": "Int32",
        "subplant_id": "Int16",
        "generator_id": "str",
        "emissions_unit_id_epa": "str",
        "boiler_id": "str",
        "operating_time_hours": "float16",
        "gross_generation_mwh": "float64",
        "steam_load_1000_lb": "float64",
        "fuel_consumed_mmbtu": "float64",
        "co2_mass_lb": "float64",
        "co2_mass_measurement_code": "category",
        "nox_mass_lb": "float64",
        "nox_mass_measurement_code": "category",
        "so2_mass_lb": "float64",
        "so2_mass_measurement_code": "category",
        "energy_source_code": "str",
        "ch4_mass_lb": "float64",
        "n2o_mass_lb": "float64",
        "fuel_consumed_for_electricity_mmbtu": "float64",
        "co2_mass_lb_for_electricity": "float64",
        "ch4_mass_lb_for_electricity": "float64",
        "n2o_mass_lb_for_electricity": "float64",
        "nox_mass_lb_for_electricity": "float64",
        "so2_mass_lb_for_electricity": "float64",
        "co2_mass_lb_adjusted": "float64",
        "ch4_mass_lb_adjusted": "float64",
        "n2o_mass_lb_adjusted": "float64",
        "nox_mass_lb_adjusted": "float64",
        "so2_mass_lb_adjusted": "float64",
        "co2_mass_lb_for_electricity_adjusted": "float64",
        "ch4_mass_lb_for_electricity_adjusted": "float64",
        "n2o_mass_lb_for_electricity_adjusted": "float64",
        "nox_mass_lb_for_electricity_adjusted": "float64",
        "so2_mass_lb_for_electricity_adjusted": "float64",
        "co2e_mass_lb": "float64",
        "co2e_mass_lb_for_electricity": "float64",
        "co2e_mass_lb_adjusted": "float64",
        "co2e_mass_lb_for_electricity_adjusted": "float64",
        "gtn_method": "category",
        "net_generation_mwh": "float64",
        "prime_mover_code": "str",
        "hourly_data_source": "category",
        "nox_control_id_eia": "str",
        "fuel_category": "str",
        "ba_code": "str",
        "ba_code_physical": "str",
        "plant_primary_fuel": "str",
        "subplant_primary_fuel": "str",
        "fuel_category_eia930": "str",
        "state": "str",
        "distribution_flag": "bool",
        "timezone": "str",
        "eia930_profile": "float64",
        "cems_profile": "float64",
        "residual_profile": "float64",
        "scaled_residual_profile": "float64",
        "shifted_residual_profile": "float64",
        "imputed_profile": "float64",
        "profile": "float64",
        "flat_profile": "float32",
        "profile_method": "str",
        "data_availability": "category",
        "equipment_tech_description": "str",
        "particulate_control_id_eia": "str",
        "so2_control_id_eia": "str",
        "nox_control_id_eia": "str",
        "mercury_control_id_eia": "str",
        "operational_status": "str",
        "hours_in_service": "float64",
        "annual_nox_emission_rate_lb_per_mmbtu": "float64",
        "ozone_season_nox_emission_rate_lb_per_mmbtu": "float64",
        "particulate_emission_rate_lb_per_mmbtu": "float64",
        "particulate_removal_efficiency_annual": "float64",
        "particulate_removal_efficiency_at_full_load": "float64",
        "so2_removal_efficiency_annual": "float64",
        "so2_removal_efficiency_at_full_load": "float64",
        "fgd_sorbent_consumption_1000_tons": "float64",
        "fgd_electricity_consumption_mwh": "float64",
        "mercury_removal_efficiency": "float64",
        "mercury_emission_rate_lb_per_trillion_btu": "float64",
        "acid_gas_removal_efficiency": "float64",
        "firing_type_1": "str",
        "firing_type_2": "str",
        "firing_type_3": "str",
        "wet_dry_bottom": "str",
    }

In [2]:
dict(sorted(dtypes_to_use.items()))

{'acid_gas_removal_efficiency': 'float64',
 'annual_nox_emission_rate_lb_per_mmbtu': 'float64',
 'ba_code': 'str',
 'ba_code_physical': 'str',
 'boiler_id': 'str',
 'cems_profile': 'float64',
 'ch4_mass_lb': 'float64',
 'ch4_mass_lb_adjusted': 'float64',
 'ch4_mass_lb_for_electricity': 'float64',
 'ch4_mass_lb_for_electricity_adjusted': 'float64',
 'co2_mass_lb': 'float64',
 'co2_mass_lb_adjusted': 'float64',
 'co2_mass_lb_for_electricity': 'float64',
 'co2_mass_lb_for_electricity_adjusted': 'float64',
 'co2_mass_measurement_code': 'category',
 'co2e_mass_lb': 'float64',
 'co2e_mass_lb_adjusted': 'float64',
 'co2e_mass_lb_for_electricity': 'float64',
 'co2e_mass_lb_for_electricity_adjusted': 'float64',
 'data_availability': 'category',
 'distribution_flag': 'bool',
 'eia930_profile': 'float64',
 'emissions_unit_id_epa': 'str',
 'energy_source_code': 'str',
 'equipment_tech_description': 'str',
 'fgd_electricity_consumption_mwh': 'float64',
 'fgd_sorbent_consumption_1000_tons': 'float64

In [5]:
boiler_firing_type = load_data.load_pudl_table(
        "boilers_eia860",
        year,
        columns=[
            "plant_id_eia",
            "boiler_id",
            "firing_type_1",
            "wet_dry_bottom",
        ],
    )

In [13]:
boiler_firing_type["wet_dry_bottom"].str.lower()

0       none
1          d
2          d
3          w
4          w
        ... 
5828       d
5829       d
5830       d
5831       d
5832       d
Name: wet_dry_bottom, Length: 5833, dtype: object